In [19]:
from classiq import *
from classiq.qmod.symbolic import floor
import numpy as np

#nth_to_last(0, list) == list[-1]
#def nth_to_last(n:int, list): 
#    return list[-1-n]

@qfunc
def tof_CT(b0:CBool, b1:CBool, ctl0:QBit, ctl1:QBit, target:QBit):
    #apply NOT to target if ctl0==b0, etc
    #keeping semantics as close to original as possible
    #within (change of basis) do CCNOT
    ctls = QArray()
    bind([ctl0,ctl1],ctls)
    within_apply(
        lambda: basischange(b0,b1,ctls[0],ctls[1]),
        lambda: control(ctrl=ctls, stmt_block=lambda: X(target))
    )
    bind(ctls,[ctl0,ctl1])
    return

#Note: this was not originally its own function, but this is very difficult to write inside a lambda.
@qfunc(generative=True)
def basischange(b0:CBool, b1:CBool, ctl0:QBit, ctl1:QBit):
    if (not b0):
        X(ctl0)
    if (not b1):
        X(ctl1)
    return



In [20]:
type Node = list[bool]

#Compute Hamming distance; rewritten slightly from Haskell for Pythonic legibility.
#Technically possible with sum(...for ...), but this is clearer.
def distance(p1:Node, p2:Node):
    return len([1 for (a,b) in zip(p1,p2) if a != b])

#Increment node to next boolstring.
#Python lists aren't linked lists, and they are mutable...
#Doing this out of place, for now.
def increment(l: Node) -> Node:
    next = l.copy()
    #flip True (1) to False (0) until we hit a False(0); then flip and break.
    for i in range (len(next)-1, -1, -1):
        if not next[i]:
            next[i] = True
            break
        else:
            next[i] = False
    return next

#Translate integer n into a length-m bitstring.
def to_node(n: int, m: int) -> Node:
    return  [((n >> (m-i-1)) % 2 == 1) for i in range(m)]

#print(increment(to_node(127, 8)))
#[True, False, False...]

In [21]:
#step to the next node in the tree, where xs are inputs and qs are ancillas.
@qfunc(generative=True)
def stepRight(n:Node, xs:QArray, qs:QArray):
    #Jesus Christ
    x0 = QBit()
    xss = QArray("xss", QBit, xs.size-1)
    bind(xs, [x0, xss])
    q0 = QBit()
    qss = QArray("qss", QBit, qs.size-1)
    bind(qs, [q0, qss])
    qs_aux = QArray("qs_aux", QBit, qs.size)
    bind([x0,qss], qs_aux)
    stepRight_aux(n, xs[1:], qs_aux)
    bind(qs_aux,[x0,qss])
    bind([q0,qss],qs)
    bind([x0,xss],xs)

@qfunc(generative=True)
def stepRight_aux(n:Node, xs:QArray, qs:QArray):
    match distance(n, increment(n)):
        case 0:
            return
        #1 -> triangle step
        case 1:
            #q0 = qs[-1]
            #q1 = qs[-2]
            CX(qs[-2],qs[-1])
            return
        #2 -> diamond step
        case 2:
            #q0 = qs[-1]
            #q1 = qs[-2]
            #q2 = qs[-3]
            #x0 = xs[-1]
            control(ctrl=qs[-2], stmt_block=X(qs[-1]))
            tof_CT(True, False, qs[-3], xs[-1], qs[-1])
            control(ctrl=qs[-3], stmt_block=X(qs[-2]))
            return
        #else recurse
        case _:
            #note: "init" means [:-1]
            #x0 = xs[-1]
            x0 = QBit()
            #xss = xs[:-1]
            xss = QArray("xss", QBit, xs.size-1)
            bind(xs,[xss,x0])
            #q0 = qs[-1]
            q0 = QBit()
            #q1 = qs[-2]
            q1 = QBit()
            #qss = qs[:-1]
            qsss = QArray("qsss", QBit, qs.size-2)
            bind(qs,[qsss,q1,q0])
            m = n[:-1]
            
            H(q0)
            TDG(q1)
            CX(x0,q1)
            T(q1)
            CX(q0,q1)
            TDG(q1)
            CX(x0,q1)
            T(q1)

            SDG(q0)
            qss = QArray("qss", QBit, qs.size-1)
            bind([qsss,q1],qss)
            stepRight_aux(m,xss,qss)
            bind(qss,[qsss,q1])

            T(q1)
            CX(x0,q1)
            T(q1)
            CX(q0,q1)
            TDG(q1)
            CX(x0,q1)
            TDG(q1)
            H(q0)

            bind([qsss,q1,q0], qs)
            bind([xss,x0],xs)


qmod = create_model(
    main,
    out_file="toftest",
)

qmod_constraint = set_constraints(
    qmod, Constraints(optimization_parameter="depth", max_width=50)
)

qprog = synthesize(qmod)

circuit_width = QuantumProgram.from_qprog(qprog).data.width
circuit_depth = QuantumProgram.from_qprog(qprog).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit_depth is {circuit_depth}")
show(qprog)